# Discussion 1: Pandas Review

This section of Discussion 1 is meant to review [Pandas](https://pandas.pydata.org/docs/), one of the most popular Python libraries for data-wrangling. It's a crucial tool in any machine learning researcher or engineer's repertoire and you will continue to use it heavily throughout the semester.

Let's start by loading the necessary packages for today's exercise, which will look at movie data from IMDb. We use the `read_csv` function to load data from the internet, but you can also use this function to load a file from your local storage.

*NOTE: The output of the cells may not be correct if run out of order, even if your code is correct. If in doubt, you can run all cells at once, which should take no more than a few seconds.*


In [1]:
import os, random, numpy as np
SEED = 189

os.environ["PYTHONHASHSEED"] = str(SEED)

random.seed(SEED)
np.random.seed(SEED)

In [2]:
import pandas as pd
import plotly.express as px

# Load the title_basics dataset from IMDb
title_basics  = pd.read_csv(
    "https://datasets.imdbws.com/title.basics.tsv.gz",
    sep="\t", compression="gzip", na_values="\\N", nrows=500000
)
# Load the title_ratings dataset from IMDb
title_ratings = pd.read_csv(
    "https://datasets.imdbws.com/title.ratings.tsv.gz",
    sep="\t", compression="gzip", na_values="\\N", nrows=500000
)

# Sort both dataframes by 'tconst' and reset the index
title_basics  = title_basics.sort_values("tconst", kind="mergesort").reset_index(drop=True)
title_ratings = title_ratings.sort_values("tconst", kind="mergesort").reset_index(drop=True)

KeyboardInterrupt: 

## Part 1: Exploration and Data Cleaning

Let's start by inspecting the `title_basics` `DataFrame`.

### Q 1.1

How many columns are in the `title_basics` `DataFrame`?

What is the data type of the startYear column? Does this make sense?


In [ ]:

# YOUR CODE HERE


### Q1.2

What is the value in 101st row of the `primaryTitle` column of the `title_basics` `DataFrame`? *HINT: Recall that* `DataFrame` *uses 0-indexing*


In [ ]:

# YOUR CODE HERE


### Q1.3

Display the first 3 rows and the last 6 rows of the `title_basics` `DataFrame` as a single `DataFrame`.


In [ ]:

# YOUR CODE HERE


### Q1.4

How many unique `titleTypes` are there in the `title_basics` `DataFrame`? Which is the most common?


In [ ]:

# YOUR CODE HERE


Now let's practice some common `DataFrame` modifications.

### Q1.5

Remove the `originalTitle` and `endYear` columns from the `title_basics` `DataFrame`. Make sure that the columns are permanently removed from the `title_basics` `DataFrame`.


In [ ]:

# YOUR CODE HERE

title_basics.head()

### Q1.6

Rename `primaryTitle` to `title` and `startYear` to `year` in the `title_basics` `DataFrame`. Make sure that the changes are reflected permanently in the `title_basics` `DataFrame`.



In [ ]:

title_basics = # YOUR CODE HERE

title_basics.head()

### Q1.7

A crucial step in most data processing pipelines for machine learning is dealing with missing or corrupted data. Often, these missing values are represented as a `NaN` (not a number).

Sometimes in the context of machine learning we'd want to estimate a value for a missing feature rather than remove that sample point entirely. Can you think of some simple ways in which we could perform that estimation?

[YOUR ANSWER HERE]

### Q1.8

Remove all rows from the `title_basics` `DataFrame` where `runtimeMinutes` or `year` is `NaN`.

In [ ]:

initial_length = title_basics.shape[0]

title_basics = # YOUR CODE HERE

final_length = title_basics.shape[0]

print(f"{initial_length - final_length} rows removed from dataframe")


### Q1.9

Change the data type of the `year` column in the `title_basics` `DataFrame` to something that makes more sense. Then confirm that the change is permanently applied.

In [ ]:

# YOUR CODE HERE


Let's practice some more basic filtering and sorting now.

### Q1.10

Extract the feature films (`titleType == "movie"`) released in 1954 from the `title_basics` `DataFrame` (save this as a new `DataFrame`, `feature_films_1954`).

In [ ]:
# YOUR CODE HERE

feature_films_1954.head()

### Q1.11

Among the feature films from 1954, which film has the longest runtime? Return its `title` and `runtimeMinutes` as a `DataFrame` extracted from the `feature_films_1954` `DataFrame`.


In [ ]:
# YOUR CODE HERE


## Part 2: Complex Modifications, Aggregations, Merging, and Plotting

Let's first modify the `title_basics` `DataFrame` so that we have one genre in for each row by duplicating rows that have multiple genres. [`df.explode`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) will be helpful for this transformation.

In [ ]:
# Split the genres string into a list of genres
title_basics['genres'] = title_basics['genres'].str.split(',')
# Explode the list of genres into separate rows
title_basics = title_basics.explode('genres')

title_basics.head()

### Q2.1

For each `genre` in the `title_basics` `DataFrame`, compute the mean runtime of feature films released since 1960.
Show the five longest-mean genres.

In [ ]:

# YOUR CODE HERE


### Q2.2

Merge the `title_ratings` `DataFrame` with the `title_basics` `DataFrame` by joining on the `tconst` column. How many titles are present in the `title_basics` `DataFrame` but not in the `title_ratings` `DataFrame`? Store the merged `DataFrame` as `merged_df`.

**Hint:** Recall that because of the genre splitting, the number of titles is not equal to the number of rows.

In [ ]:
n_titles_basics = ... # YOUR CODE HERE

merged_df = ... # YOUR CODE HERE
print(merged_df.head())

n_titles_merged = ... # YOUR CODE HERE

print(f"\nNumber of titles in basics but not in ratings: {n_titles_basics - n_titles_merged}")

### Q2.3

Using the `merged_df` `DataFrame` and plotly express, create an interactive scatter plot of the `runtimeMinutes` vs. `numVotes` for movies in the `merged_df` `DataFrame`.
Color the points by the `year` of the movie and add a title and axis labels to the plot. Also, make sure the movie title is visible when hovering over the
data points.

**Note:** To make the data easier to visualize, we take a sample of just 2000 movies. That's why you may not see your favorites on this plot. It's important not to change the random state as you'll end up getting different results for the following questions.

In [ ]:
sampled_df = merged_df[merged_df['titleType'] == 'movie'].sample(n=2000, random_state=SEED)

# YOUR CODE HERE


### Q2.4

Describe any trends you see in the plot above.

[YOUR ANSWER HERE]

### Q2.5
Which two movies in the plot received the most votes and had the longest runtime, respectively? When were they each released?

[YOUR ANSWER HERE]

# Part 3: Finding the perfect movie

Aakarsh has spent his whole summer brainrotting and doomscrolling, so now his attention span is COOKED. He wants to pick a movie to watch tonight but wants to make sure it isn't so long he gets bored. He decides to construct a Brainrot Score (BRS) to help him find the perfect movie:

$$BRS = \frac{\text{averageRating}}{\sqrt{\text{runtimeMinutes}}}$$

He also wants to make sure the following criteria hold:
- The title should be a *movie* made in 1980 or later.
- It must have at least 10000 votes.
- It must be in the `History`, `Thriller`, or `Comedy` genres.

Can you help Aakarsh out by finding the 3 best movies by BRS in each of his preferred genres?

In [ ]:
# YOUR CODE HERE
